# Download Data

# Download Youtube Audio

In [ ]:
from pytube import YouTube

# List of YouTube video URLs for guided Jhana meditations
video_urls = [
    "https://www.youtube.com/watch?v=9qXu-_sHy5Q",  # A Guided Meditation on the Breath and First Jhana - Ajahn Lee
    "https://www.youtube.com/watch?v=a7wTOKnJeds",  # The Four Jhanas - A Guided Meditation - Ajahn Lee
    "https://www.youtube.com/watch?v=0K5ypXyF3dY",  # How to Jhana — with Michael Taft
    "https://www.youtube.com/watch?v=pkwPIL4ZNPg",  # Guided Meditation Through the Four Jhanas
    "https://www.youtube.com/watch?v=a0UUkLme5UE",  # Bhante Gunaratana Jhana DVD 01A - Guided Meditation
    "https://www.youtube.com/watch?v=0FhQD9pfJL4",  # Meditation With Focus On Jhana - Ven Ajahn Brahm
    "https://www.youtube.com/watch?v=Rs9ibiPy-5s",  # Guided Meditation - Attaining Rupa and Arupa Jhanas
    "https://www.youtube.com/watch?v=9UaGROTkkfY",  # Guided Sunrise Meditation to the First Jhana
    "https://www.youtube.com/watch?v=dHuYtz0zLUQ",  # Advanced Breath Meditation and Jhana
    "https://www.youtube.com/watch?v=y2h54_Tcl9w"   # GUIDED MEDITATION 1st Jhana, using Fire Kasina
]

for url in video_urls:
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True).first()
    filename = yt.title.replace("/", "-")  # Replace any slashes in title to avoid path issues
    output_path = "../data/input/audio/guided_meditations/youtube"  # Adjust if necessary
    stream.download(output_path=output_path, filename=f"{filename}.mp3")

print("Download completed.")


Download completed.


# Scrape the Web

# dharmaseed.org

Dharmaseed.org is a website that provides a large collection of Dharma talks. The website has a search function that allows users to search for talks by teacher, topic, or date. The search results are displayed in a list with the title, teacher, date, and duration of the talk. The website also provides a link to download the audio file of the talk.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import os

def download_mp3(mp3_url, save_path):
    try:
        response = requests.get(mp3_url, stream=True)
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Error downloading {mp3_url}: {str(e)}")

def scrape_and_download(url, base_save_path):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to access {url}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', string='Download')  # Use 'string' instead of 'text'

    for link in links:
        # Check if the URL is absolute. If not, prepend the base URL.
        mp3_url = link['href']
        if mp3_url.startswith('/'):
            mp3_url = requests.compat.urljoin(url, mp3_url)  # Ensure mp3_url is an absolute URL

        file_name = mp3_url.split('/')[-1]
        save_path = os.path.join(base_save_path, file_name)
        download_mp3(mp3_url, save_path)
        time.sleep(1)  # Respectful gap between downloads

base_save_path = "../data/input/audio/guided_meditations/audiodharma_dharmaseed/"
if not os.path.exists(base_save_path):
    os.makedirs(base_save_path)

urls = [
    "https://dharmaseed.org/talks/?search=jhana&sort=-rec_date&page_items=100",
    "https://www.audiodharma.org/playables/search?query=jhana"
]

for url in urls:
    scrape_and_download(url, base_save_path)
    time.sleep(5)  # Respectful gap between scraping different pages
